# Dependencies

In [200]:
%%capture
!pip install pandas
!pip install openai
!pip install tensorflow
!pip install scikit-learn
!pip install protobuf==3.20.*
!pip install mediapipe==0.9.0.1

In [203]:
import os
import random
import matplotlib
import numpy as np
from itertools import chain
from collections import deque
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display, Image

import cv2
import openai
import pandas as pd
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from mediapipe.framework.formats import landmark_pb2
from sklearn.utils import check_random_state as sklear_set_seed

In [3]:
!python --version
print("TensorFlow v" + tf.__version__)
print("Mediapipe v" + mp.__version__)

Python 3.9.13
TensorFlow v2.14.0
Mediapipe v0.9.0.1


In [32]:
seed = 42
random.seed(seed)
np.random.seed(seed)
cv2.setRNGSeed(seed)
sklear_set_seed(seed)
tf. random.set_seed(seed)

# Prepare dataset

In [34]:
dataset_df = pd.read_csv("fehervali_lamp_night_webcam.csv", dtype={63: str}, header=None)
dataset_df = dataset_df[dataset_df[63].notna()]
print(len(dataset_df))
dataset_df.head(3)

79054


0         1             2         3         4         5         6   \
0  0.282440  0.600049 -1.870000e-07  0.315119  0.568314 -0.020261  0.341418   
1  0.306540  0.464684 -2.530000e-07  0.345548  0.433487 -0.015010  0.376949   
2  0.312548  0.466279 -2.500000e-07  0.351508  0.435728 -0.017326  0.383225   

         7         8         9   ...        54        55        56        57  \
0  0.497320 -0.028145  0.345182  ...  0.245077  0.454705 -0.029709  0.256525   
1  0.352773 -0.019309  0.383595  ...  0.273192  0.294298 -0.030596  0.283822   
2  0.355989 -0.022386  0.390655  ...  0.279703  0.295002 -0.031225  0.289348   

         58        59        60        61        62  63  
0  0.503494 -0.027200  0.265023  0.532601 -0.019086   a  
1  0.349954 -0.026084  0.290784  0.386303 -0.016583   a  
2  0.351177 -0.027367  0.296098  0.386388 -0.017859   a  

[3 rows x 64 columns]

As the data cosists of sequential frames many of them are similar, so only every n frames are kept.

In [35]:
n = 10
dataset_df = dataset_df[dataset_df.index % n == 0]

In [36]:
category_counts = dataset_df[63].value_counts()
plt.figure(figsize=(8, 6))
category_counts.plot(kind='bar')
plt.xlabel('Categories')
plt.ylabel('Count')
plt.title(f'Distribution of Categories')
plt.show()

In [52]:
PATH_TRAIN_DS = "shuffled_dataset_train.csv"
PATH_VALID_DS = "shuffled_dataset_train.csv"
shuffled_df = dataset_df.sample(frac=1).reset_index(drop=True)

train_df, valid_df = train_test_split(shuffled_df, train_size=0.8, shuffle=True)

train_df.to_csv(PATH_TRAIN_DS, index=False)
valid_df.to_csv(PATH_VALID_DS, index=False)

# Load dataset

In [67]:
sorted_cats = sorted(set(dataset_df[63].apply(lambda c: str(c))))
cat_to_num = {sign : i for i, sign in enumerate(sorted_cats)}
num_to_cat = {i : sign for sign, i in cat_to_num.items()}

tf_cat_to_num = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(cat_to_num.keys()),
        values=list(cat_to_num.values()),
    ),
    default_value=tf.constant(-1),
    name="signs"
)

## Augmentation

Data looks like this: x1,y1,z1, x2,y2,z2 ....

In [148]:
def augment_lm(flat_xyz,
               flip_prob=0.3,
               zoom_prob=0.3,
               affine_prob=0.3,
               scale=(0.8, 1.2),
               shear=(-0.15, 0.15),
               shift=(-0.1, 0.1),
               degree=(-30, 30),
               zoom=(0.8, 1.2)):
    
    # Reshape flat data to [num_samples, 3]
    xyz = tf.reshape(flat_xyz, [21, 3]) # 21 coords for the hands
    
    # 1. Flip x coordinates with given probability
    if tf.random.uniform(()) < flip_prob:
        xyz = tf.stack([1-xyz[:, 0], xyz[:, 1], xyz[:, 2]], axis=1)
    
    # 2. Apply zoom with given probability
    if tf.random.uniform(()) < zoom_prob:
        zx = zy = tf.random.uniform((), *zoom)
    else:
        zx, zy = 1.0, 1.0
    
    # 3. Affine transformations
    if tf.random.uniform(()) < affine_prob:
        # Random scale factors for x and y
        sx = tf.random.uniform((), *scale)
        sy = tf.random.uniform((), *scale)
        sx *= zx
        sy *= zy

        # Random shear factors for x and y
        shx = tf.random.uniform((), *shear)
        shy = tf.random.uniform((), *shear)

        # Random shift for x and y
        tx = tf.random.uniform((), *shift)
        ty = tf.random.uniform((), *shift)

        # Random rotation in radians
        pi_on_180 = 0.017453292519943295
        theta = tf.random.uniform((), degree[0]*pi_on_180, degree[1]*pi_on_180)
        c, s = tf.cos(theta), tf.sin(theta)

        # Constructing the affine transformation matrix
        matrix = tf.convert_to_tensor([
            [sx*c + shx*s, -sy*s + shx*c, tx],
            [sx*s + shy*c, sy*c + shy*s, ty]
        ])

        # Apply transformation to x and y
        xy = tf.matmul(xyz[:, :2], tf.transpose(matrix[:, :2])) + matrix[:, 2]
    else:
        xy = xyz[:, :2]
    
    # Combining transformed x and y with original z
    transformed_xyz = tf.concat([xy, tf.expand_dims(xyz[:, 2], axis=-1)], axis=1)
    
    # Flatten back to original shape
    flat_transformed = tf.reshape(transformed_xyz, [-1])
    
    return flat_transformed

In [149]:
# Sample test
flat_xyz_data = tf.constant([0.1, 0.3, 0.4, 0.5, 0.6, 0.7])
transformed = spatial_random_affine(flat_xyz_data)
print(transformed)

tf.Tensor([0.08281422 0.325535   0.4        0.43096402 0.8735771  0.7       ], shape=(6,), dtype=float32)


## Preprocessing

In [150]:
def tf_nan_mean(x, axis=0, keepdims=False):
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis, keepdims=keepdims) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis, keepdims=keepdims)

def tf_nan_std(x, center=None, axis=0, keepdims=False):
    if center is None:
        center = tf_nan_mean(x, axis=axis,  keepdims=True)
    d = x - center
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis, keepdims=keepdims))

def preprocess_lms(lm):
    mean = tf_nan_mean(lm)
    std = tf_nan_std(lm)
    lm = (lm - mean)/std
    lm = tf.where(tf.math.is_nan(lm),tf.constant(0.,lm.dtype),lm)
    return lm

def preprocess_cat(cat):
    return tf_cat_to_num.lookup(cat)

def preprocess_csv(line, augment=False):
    columns = tf.strings.split(line, ',')
    features = tf.strings.to_number(columns[:-1], out_type=tf.float32)
    
    if augment:
        features = augment_lm(features)
        
    return preprocess_lms(features), preprocess_cat(columns[-1])

In [151]:
def get_dataset(csv_path, *, batch_size=1, drop_remainder=False, shuffle_size=None, repeat=False, cache=False, augment=False):
    ds = tf.data.TextLineDataset(csv_path).skip(1) # skip header
    ds = ds.map(lambda x: preprocess_csv(x, augment=augment), tf.data.AUTOTUNE)
    
    if repeat: 
        ds = ds.repeat()
    
    if shuffle_size:
        ds = ds.shuffle(shuffle_size)
        options = tf.data.Options()
        options.experimental_deterministic = (False)
        ds = ds.with_options(options)
        
    if batch_size > 1:
        ds = ds.batch(batch_size)
    
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    if cache:
        ds = ds.cache()
    
    return ds

batch_size = 256

train_ds = get_dataset(PATH_TRAIN_DS, batch_size=batch_size, shuffle_size=4*batch_size, augment=True)
valid_ds = get_dataset(PATH_VALID_DS, batch_size=batch_size)

In [152]:
for batch in train_ds:
    print(batch)
    break

(<tf.Tensor: shape=(256, 63), dtype=float32, numpy=
array([[ 0.91803676,  0.04994599, -1.1481365 , ...,  1.0608692 ,
         0.2573768 , -1.2382442 ],
       [ 0.43908828,  1.9541461 , -0.99697226, ...,  0.38656712,
        -0.45505682, -1.3197205 ],
       [-0.25387034,  1.6339612 , -0.9422369 , ..., -0.32827696,
         1.5115246 , -1.0375955 ],
       ...,
       [-0.15190245,  1.5807458 , -0.6785015 , ..., -0.5493588 ,
         1.6863092 , -0.8825209 ],
       [-0.05439661,  2.4683104 , -0.81832457, ..., -0.54877067,
         0.7105839 , -1.0772018 ],
       [ 0.7113025 ,  1.34161   , -1.281158  , ...,  0.7795377 ,
         1.0745475 , -1.4045508 ]], dtype=float32)>, <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([24,  4, 19,  2, 26, 10,  1,  6, 25, 12, 19, 32, 29, 17,  8, 30,  7,
       22, 24, 32, 26, 18,  1,  9, 13, 11, 27,  1, 11,  5, 11,  9, 19, 20,
       23, 20, 17, 18, 30, 30, 11, 16, 24, 14, 26,  1, 32, 30, 21, 14, 20,
       18, 24, 15, 19, 17,  0, 27, 16, 24,  3, 

# Models

## Dense model

In [153]:
model = keras.Sequential([
    layers.Input(shape=(63,)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(sorted_cats), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               8192      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 33)                2145      
                                                                 
Total params: 18593 (72.63 KB)
Trainable params: 18593 (72.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## CNN model

In [167]:
model = keras.Sequential([
    layers.Input(shape=(63,)),
    layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=-1)),
    layers.Conv1D(4, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(sorted_cats), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 63, 1)             0         
                                                                 
 conv1d_5 (Conv1D)           (None, 61, 4)             16        
                                                                 
 flatten_2 (Flatten)         (None, 244)               0         
                                                                 
 dense_15 (Dense)            (None, 128)               31360     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)               

In [171]:
%%time
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=30,
          verbose = 2)

Epoch 1/30
7/7 - 0s - loss: 0.7167 - accuracy: 0.7592 - val_loss: 0.4637 - val_accuracy: 0.8774 - 311ms/epoch - 44ms/step
Epoch 2/30
7/7 - 0s - loss: 0.6698 - accuracy: 0.7661 - val_loss: 0.4582 - val_accuracy: 0.8900 - 296ms/epoch - 42ms/step
Epoch 3/30
7/7 - 0s - loss: 0.6903 - accuracy: 0.7459 - val_loss: 0.4560 - val_accuracy: 0.8957 - 294ms/epoch - 42ms/step
Epoch 4/30
7/7 - 0s - loss: 0.6841 - accuracy: 0.7668 - val_loss: 0.4745 - val_accuracy: 0.8666 - 318ms/epoch - 45ms/step
Epoch 5/30
7/7 - 0s - loss: 0.6732 - accuracy: 0.7649 - val_loss: 0.4449 - val_accuracy: 0.8938 - 297ms/epoch - 42ms/step
Epoch 6/30
7/7 - 0s - loss: 0.6611 - accuracy: 0.7693 - val_loss: 0.4405 - val_accuracy: 0.8919 - 271ms/epoch - 39ms/step
Epoch 7/30
7/7 - 0s - loss: 0.6823 - accuracy: 0.7566 - val_loss: 0.4480 - val_accuracy: 0.8729 - 290ms/epoch - 41ms/step
Epoch 8/30
7/7 - 0s - loss: 0.6376 - accuracy: 0.7794 - val_loss: 0.4373 - val_accuracy: 0.8818 - 274ms/epoch - 39ms/step
Epoch 9/30
7/7 - 0s - lo

# Testing

## Util to work with video feeds using mediapipe hands

In [174]:
WEBCAM = 0

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def video_loop_mp_hands(source, process_data_func):
    video = cv2.VideoCapture(source)
    display_handle=display(None, display_id=True)
    try:
        with mp_hands.Hands(model_complexity=1,max_num_hands=1,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
            while True:
                _, image = video.read()
    
                if image is None:
                    break

                image. flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image)

                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                image = cv2.flip(image, 1)
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks: # 1 hand only
                        # Draw landmarks
                        mp_drawing.draw_landmarks(
                            image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())
        
                        # Get bounding box for the hand
                        lms = hand_landmarks.landmark
                        x_coords = [lm.x for lm in lms]
                        y_coords = [lm.y for lm in lms]
                        min_x, max_x = int(min(x_coords) * image.shape[1]), int(max(x_coords) * image.shape[1])
                        min_y, max_y = int(min(y_coords) * image.shape[0]), int(max(y_coords) * image.shape[0])

                        image = cv2.flip(image, 1)
                        # Draw the bounding box around the hand
                        cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)  # RGB green
                
                        data = []
                        for lm in lms:
                            data.extend([lm.x, lm.y, lm.z])

                        # Get prediction and probability from function
                        pred, prob = process_data_func(data)

                        image = cv2.flip(image, 1)
                        # Display prediction and probability near the bounding box
                        label = f"{pred}: {prob:.2f}%"
                        cv2.putText(image, label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    
                
                _, image = cv2.imencode('.jpeg', image)
                display_handle.update(Image(data=image.tobytes()))
    except KeyboardInterrupt:
        pass
    finally:
        video.release()
        display_handle.update(None)

## Predict every frame model for isolated signs

While this model performs well, but in certain positions and during changing poses, random predictions might happen.

In [175]:
def process_frame(frame):
    x = preprocess_lms(frame)
    probs = model(np.array([x]))
    prob = np.max(probs, axis=1)[0]
    pred = np.argmax(probs)
    pred = num_to_cat[pred]
    return pred, prob

video_loop_mp_hands(WEBCAM, process_frame)

None

## Buffered model for isolated signs

In [176]:
class BufferedModel:
    def __init__(self, model, preprocess_fn, *, confidence, buffer_size):
        self.model = model
        self.preprocess_fn = preprocess_fn
        self.buffer = deque(maxlen=buffer_size)
        self.confidence_threshold = int(buffer_size*confidence)
        self.buffer_size = buffer_size

    def process_frame(self, frame):
        x = self.preprocess_fn(frame)
        probs = self.model(np.array([x]))
        prob = np.max(probs, axis=1)[0]
        pred = np.argmax(probs)
        
        self.buffer.append(pred)
        buffered_pred, count = Counter(self.buffer).most_common(1)[0]
        if count >= self.confidence_threshold:
            return num_to_cat[buffered_pred], count/self.buffer_size

        return "unknown", 0.
            
buffered_model = BufferedModel(model, preprocess_lms, confidence=0.7, buffer_size=10)
video_loop_mp_hands(WEBCAM, lambda data: buffered_model.process_frame(data))

None

## Continuous model with signing detection

For this model the mediapipe holistic model is needed, gather more pose information than simply the 2 hands.

### Handle mediapipe holistic model

In [193]:
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

def draw_landmarks_on_image(image, results):
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style()
    )
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style()
    )
    return image

def video_loop_mp_holistics(source, process_data_func):
    video = cv2.VideoCapture(source)
    display_handle=display(None, display_id=True)
    try:
        with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
            while True:
                _, frame = video.read()
    
                if frame is None:
                    break
    
                #image = cv2.resize(frame, (360, 240))
                image=frame
    
                # To improve performance, optionally mark the image as not writeable to pass by reference.
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)

                # Run the model
                process_data_func(results)
    
                # Draw landmark annotation on the image.
                image = draw_landmarks_on_image(image, results)
    
                image = cv2.flip(image, 1)
                _, image = cv2.imencode('.jpeg', image)
                display_handle.update(Image(data=image.tobytes()))
    except KeyboardInterrupt:
        pass
    finally:
        video.release()
        display_handle.update(None)

### Signing detection model

In [194]:
# Pose coordinates for hand movement.
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

def extract_for_signing_detection(res):
    # Extract specific pose landmarks if available
    px = []
    py = []
    pz = []
    if res.pose_landmarks:
        for i in POSE:
            lm = res.pose_landmarks.landmark[i]
            px.append(lm.x)
            py.append(lm.y)
            pz.append(lm.z)
    else:
        px = [0.0]*len(POSE)
        py = [0.0]*len(POSE)
        pz = [0.0]*len(POSE)

    # Extract left hand landmarks if available
    lx = []
    ly = []
    lz = []
    if res.left_hand_landmarks:
        for lm in res.left_hand_landmarks.landmark:
            lx.append(lm.x)
            ly.append(lm.y)
            lz.append(lm.z)
    else:
        lx = [0.0]*21
        ly = [0.0]*21
        lz = [0.0]*21

    # Extract right hand landmarks if available
    rx = []
    ry = []
    rz = []
    if res.right_hand_landmarks:
        for lm in res.right_hand_landmarks.landmark:
            rx.append(lm.x)
            ry.append(lm.y)
            rz.append(lm.z)
    else:
        rx = [0.0]*21
        ry = [0.0]*21
        rz = [0.0]*21

    return list(chain(rx, lx, px, ry, ly, py, rz, lz, pz))

# Only load once
signing_detection_model = tf.saved_model.load("signing_detection_model")

class SigningDetectionModel:
    def __init__(self):
        self.signing_detection_model_input = list(np.zeros((15, 156)))

    def is_signing(self, inp_lm):
        self.signing_detection_model_input.pop(0)
        self.signing_detection_model_input.append(inp_lm)
        return signing_detection_model.predict(self.signing_detection_model_input)["result"].numpy() == 1

class BufferedSigningDetectionModel:
    def __init__(self, buffer_len=5, confidence_number=3):
        self.signing_detection_model_input = list(np.zeros((15, 156)))
        self.signing_detector_buffer = deque(maxlen=buffer_len)
        self.confidence_number = confidence_number 

    def is_signing(self, inp_lm):
        self.signing_detection_model_input.pop(0)
        self.signing_detection_model_input.append(inp_lm)
        pred = signing_detection_model.predict(self.signing_detection_model_input)["result"].numpy()
        self.signing_detector_buffer.append(pred)
        buffered_pred, count = Counter(self.signing_detector_buffer).most_common(1)[0]
        if count >= self.confidence_number:
            return buffered_pred == 1
        else:
            return False

In [199]:
def extract_for_static_fingerspelling(res):
    # Extract right hand landmarks if available
    data = []
    if res.right_hand_landmarks:
        for lm in res.right_hand_landmarks.landmark:
            data.append(lm.x)
            data.append(lm.y)
            data.append(lm.z)
    elif res.left_hand_landmarks:
        for lm in res.left_hand_landmarks.landmark:
            data.append(lm.x)
            data.append(lm.y)
            data.append(lm.z)
    else:
        data = [0.0]*63
        
    return data

class ContinuousModelWithSigningDetection:
    def __init__(self):
        self.signing_detection_model = BufferedSigningDetectionModel()
        self.buffered_model = BufferedModel(model, preprocess_lms, confidence=0.7, buffer_size=10)
        self.last_pred = None

    def process_result(self, mp_result):
        sd_inp = extract_for_signing_detection(mp_result)
        if self.signing_detection_model.is_signing(sd_inp):
            print("*", end="")
            pred, prob = self.buffered_model.process_frame(extract_for_static_fingerspelling(mp_result))
            if self.last_pred != pred and pred != "unknown":
                self.last_pred = pred
                print(pred, end="")
                return
        else:
            print("-", end="")
            self.last_pred = None

m = ContinuousModelWithSigningDetection()

video_loop_mp_holistics(WEBCAM, lambda mp_res: m.process_result(mp_res))

None

--------------------------------------------------------------------------*******a****************************b*****************************c****************************************l***********************************************r*************************************g****************************************-------------------------------------------------------------------------------------------------------------------------------------------------------------*******h***********************************5****************b*****************************t**********-------*t********y********************************t****************************y*********************t*******************y**************------------------------------

### LLm improvement

In [204]:
key = os.environ.get('OPEN_AI_API_KEY')
if key is not None:
    openai.api_key = key
else:
    print("Error: Please set a valid api key!")

In [205]:
def correct_output(pred):
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", 
       messages=[
        {"role": "system", "content": "You are a machine that tries to correct the output of a fingerspelling recognition model. Some letters might be missing, but it's also possible that the given text has extra characters. The model will always give i instead of j and z also doesn't work. Only reply the corrected text."},
        {"role": "user", "content": "lebra"},
        {"role": "system", "content": "zebra"},
        {"role": "user", "content": "beark"},
        {"role": "system", "content": "bear"},
        {"role": "user", "content": "iagudar"},
        {"role": "system", "content": "jaguar"},
        {"role": "user", "content": pred},
      ]
    )
    
    return completion["choices"][0]["message"]["content"]

In [208]:
correct_output("iackdl")

'jackal'

In [209]:
correct_output("loom") # what happens if both are acceptable

'zoom'